# Parametric Study of 2D Transient Stokes Flow in a Rectangular Channel

**Objective:** The goal of this notebook is to perform a parametric study on the 2D transient Stokes flow model for an open rectangular channel. We will investigate the impact of mesh refinement and different implicit time-integration schemes on the solver's performance and statistics.

The implementation is based on validated code using `Ferrite.jl` for the finite element formulation and `DifferentialEquations.jl` for the time integration.

##  Section 1: Required Packages

In [1]:
using BlockArrays
using LinearAlgebra
using UnPack
using SparseArrays
using Ferrite
using OrdinaryDiffEq
using DifferentialEquations
using Plots
using WriteVTK
using DataFrames
using Printf

## Section 2: Problem Definition

We solve the transient Stokes equations for an incompressible fluid in a 2D domain $\Omega$.

#### Momentum Equation
$$ \frac{\partial \mathbf{u}}{\partial t} - \mu \nabla^2 \mathbf{u} + \nabla p = \mathbf{0} \quad \text{in } \Omega $$

#### Continuity Equation (Incompressibility)
$$ \nabla \cdot \mathbf{u} = 0 \quad \text{in } \Omega $$

where $\mathbf{u}$ is the velocity, $p$ is the pressure, and $\mu$ is the viscosity.

### Boundary Conditions for the Open Channel:
- **Inlet (left):** A time-ramped parabolic velocity profile.
- **Walls (top/bottom):** No-slip condition ($\mathbf{u} = \mathbf{0}$).
- **Outlet (right):** Zero pressure condition ($p=0$).

## Section 3: Core Assembly Functions

These are the validated functions from the reference notebooks for assembling the stiffness and mass matrices. We use them directly to ensure our study is based on a correct physical model.

In [2]:
function assemble_mass_matrix!(cellvalues_v::CellValues, M::SparseMatrixCSC, dh::DofHandler)
    n_basefuncs_v = getnbasefunctions(cellvalues_v)
    Mₑ = zeros(n_basefuncs_v, n_basefuncs_v)
    
    # We only need the velocity-velocity block
    u_dofs = dof_range(dh, :u)
    
    mass_assembler = start_assemble(M)
    for cell in CellIterator(dh)
        fill!(Mₑ, 0)
        Ferrite.reinit!(cellvalues_v, cell)
        
        for q_point in 1:getnquadpoints(cellvalues_v)
            dΩ = getdetJdV(cellvalues_v, q_point)
            for i in 1:n_basefuncs_v
                φᵢ = shape_value(cellvalues_v, q_point, i)
                for j in 1:n_basefuncs_v
                    φⱼ = shape_value(cellvalues_v, q_point, j)
                    Mₑ[i, j] += (φᵢ ⋅ φⱼ) * dΩ
                end
            end
        end
        
        # Assemble only into the velocity part of the global matrix
        assemble!(mass_assembler, celldofs(cell)[u_dofs], Mₑ)
    end
    return M
end

function assemble_stokes_matrix!(K, dh, cvu, cvp, viscosity)
    assembler = start_assemble(K)
    ke = zeros(ndofs_per_cell(dh), ndofs_per_cell(dh))
    
    range_u = dof_range(dh, :u)
    ndofs_u = length(range_u)
    range_p = dof_range(dh, :p)
    ndofs_p = length(range_p)
    
    # Pre-allocate buffers
    ∇ϕᵤ = Vector{Tensor{2,2,Float64,4}}(undef, ndofs_u)
    divϕᵤ = Vector{Float64}(undef, ndofs_u)
    ϕₚ = Vector{Float64}(undef, ndofs_p)
    
    for cell in CellIterator(dh)
        Ferrite.reinit!(cvu, cell)
        Ferrite.reinit!(cvp, cell)
        fill!(ke, 0)
        
        for qp in 1:getnquadpoints(cvu)
            dΩ = getdetJdV(cvu, qp)
            
            for i in 1:ndofs_u
                ∇ϕᵤ[i] = shape_gradient(cvu, qp, i)
                divϕᵤ[i] = shape_divergence(cvu, qp, i)
            end
            for i in 1:ndofs_p
                ϕₚ[i] = shape_value(cvp, qp, i)
            end
            
            # u-u block (viscosity)
            for (i, I) in pairs(range_u), (j, J) in pairs(range_u)
                ke[I, J] += viscosity * (∇ϕᵤ[i] ⊡ ∇ϕᵤ[j]) * dΩ
            end
            
            # u-p and p-u blocks (divergence/gradient)
            for (i, I) in pairs(range_u), (j, J) in pairs(range_p)
                ke[I, J] -= divϕᵤ[i] * ϕₚ[j] * dΩ
                ke[J, I] -= divϕᵤ[i] * ϕₚ[j] * dΩ
            end
        end
        assemble!(assembler, celldofs(cell), ke)
    end
    return K
end

assemble_stokes_matrix! (generic function with 1 method)

## Section 4: Simulation Runner Function

To facilitate the parametric study, we create a single function, `run_simulation`, that encapsulates the entire process for a given set of parameters. This function will:
1. Take mesh size, solver type, and tolerances as input.
2. Perform all setup steps: mesh generation, FE space definition, matrix assembly.
3. Define and solve the `ODEProblem`.
4. Return the solution object, which contains performance statistics.

In [3]:
struct RHSParams
    K::SparseMatrixCSC
    ch::ConstraintHandler
    u_buffer::Vector{Float64} # Buffer to hold the full solution vector
end

# The ODE function (du/dt = f(u,p,t))
# We solve M*du/dt = -K*u, so f(u,p,t) = -K*u
function stokes!(du, u_ode, p::RHSParams, t)
    @unpack K, ch, u_buffer = p
    
    # The input u_ode from the solver contains only free dofs.
    # We need to apply boundary conditions to get the full state vector.
    u_buffer .= u_ode
    update!(ch, t) # Update for time-dependent BCs
    apply!(u_buffer, ch)
    
    # Compute the residual: du = -K * u
    mul!(du, K, u_buffer)
    du .*= -1.0
    
    return
end

# Jacobian of the ODE function (J = d(f)/du = -K)
# The Jacobian is constant for this linear problem.
function stokes_jac!(J, u_ode, p::RHSParams, t)
    # The Jacobian is simply -K, with BCs applied.
    # We pre-calculate this and store it in the ODEFunction jac_prototype.
    # This function just returns the pre-built Jacobian.
    return J
end

# Custom limiter to apply Dirichlet BCs at each step
function ferrite_limiter!(u, _, p, t)
    update!(p.ch, t)
    apply!(u, p.ch)
end

# Custom norm to calculate error only on free DOFs
struct FreeDofErrorNorm
    ch::ConstraintHandler
end
(fe_norm::FreeDofErrorNorm)(u, t) = norm(u[fe_norm.ch.free_dofs]) / sqrt(length(fe_norm.ch.free_dofs))

In [4]:
function run_simulation(nels::Tuple{Int,Int}, timestepper, T_end::Float64, dt0::Float64, abstol::Float64, reltol::Float64; viscosity=1e3)
    
    # 1. MESH and GEOMETRY
    H = 0.25
    L = 4 * H
    grid = generate_grid(Quadrilateral, nels, Vec((0.0, 0.0)), Vec((L, H)))
    
    # 2. FINITE ELEMENT SPACES (Q2/Q1 Taylor-Hood)
    dim = 2
    degree = 1
    ipu = Lagrange{RefQuadrilateral, degree + 1}()^dim
    ipp = Lagrange{RefQuadrilateral, degree}()
    
    dh = DofHandler(grid)
    add!(dh, :u, ipu)
    add!(dh, :p, ipp)
    close!(dh)
    
    qr = QuadratureRule{RefQuadrilateral}(2 * degree + 1)
    ipg = Lagrange{RefQuadrilateral, 1}()
    cvu = CellValues(qr, ipu, ipg)
    cvp = CellValues(qr, ipp, ipg)
    
    # 3. BOUNDARY CONDITIONS
    ch = ConstraintHandler(dh)
    vmax = 1.0
    vin(t) = min(t * vmax, vmax) # Ramped velocity
    parabolic_inflow(x, t) = Vec((vin(t) * x[2] * (H - x[2]) / (H^2 / 4), 0.0))
    
    add!(ch, Dirichlet(:u, getfacetset(grid, "left"), parabolic_inflow))
    add!(ch, Dirichlet(:u, union(getfacetset(grid, "top"), getfacetset(grid, "bottom")), (x,t) -> [0,0]))
    add!(ch, Dirichlet(:p, getfacetset(grid, "right"), (x,t) -> 0))
    close!(ch)
    
    # 4. MATRIX ASSEMBLY
    K = allocate_matrix(dh, ch)
    M = allocate_matrix(dh, ch)
    
    assemble_stokes_matrix!(K, dh, cvu, cvp, viscosity)
    assemble_mass_matrix!(cvu, M, dh)
    
    # 5. ODE PROBLEM SETUP
    # The Jacobian of our ODE function is -K. We apply BCs to it once.
    J = -K
    apply!(J, ch)
    jac_sparsity = sparse(J)
    
    # Set up the ODE function with the constant Jacobian
    rhs = ODEFunction(stokes!, mass_matrix=M, jac=stokes_jac!, jac_prototype=jac_sparsity)
    
    # Initial condition (zero velocity)
    u0 = zeros(ndofs(dh))
    update!(ch, 0.0)
    apply!(u0, ch)
    
    # Parameters for the ODE function
    params = RHSParams(K, ch, copy(u0))
    
    problem = ODEProblem(rhs, u0, (0.0, T_end), params)
    
    # 6. SOLVE
    # Set the step limiter for the chosen timestepper
    # We need to create a new stepper instance to set the limiter
    stepper_instance = deepcopy(timestepper)
    if hasproperty(stepper_instance, :step_limiter!)
        stepper_instance.step_limiter! = ferrite_limiter!
    end
    
    sol = solve(problem, stepper_instance, dt=dt0,
                adaptive=true, 
                abstol=abstol, reltol=reltol,
                progress=false, # Disable progress bar for clean output
                internalnorm=FreeDofErrorNorm(ch),
                d_discontinuities=[1.0] # Discontinuity at t=1 when ramp finishes
               )
    
    return sol, ndofs(dh)
end

run_simulation (generic function with 1 method)

## Section 5: Parametric Study Configuration

Here we define the different configurations we want to test. We will vary:
1. **Mesh Size:** From coarse to fine, to see how performance scales with the number of degrees of freedom (DOFs).
2. **Time Stepper:** A selection of implicit solvers from `DifferentialEquations.jl` suitable for stiff, differential-algebraic equations (DAEs).
3. **Tolerances:** We'll test both low and high accuracy settings to see the trade-off between speed and precision.

In [ ]:
# General simulation parameters
T_end = 2.0  # Total simulation time
dt0 = 0.01   # Initial time step

# 1. Define Mesh Sizes
mesh_configs = [
    (name="Coarse (10x4)", nels=(10, 4)),
    (name="Medium (20x5)", nels=(20, 5)),
    (name="Fine (40x10)",   nels=(40, 10))
]

# 2. Define Time Steppers
# Note: autodiff=false is crucial for these manually defined Jacobian systems.
stepper_configs = [
    (name="Rodas5P", stepper=Rodas5P(autodiff=false)),
    (name="ROS34PW1a", stepper=ROS34PW1a(autodiff=false)),
    (name="KenCarp4", stepper=KenCarp4(autodiff=false))
]

# 3. Define Tolerances
tolerance_configs = [
    (name="Low Accuracy", abstol=1e-3, reltol=1e-3),
    (name="High Accuracy", abstol=1e-5, reltol=1e-5)
]

# Array to store results
results = []

Any[]

## Section 6: Executing the Study

We now loop through all combinations of the configured parameters, run the simulation, and collect the statistics.

In [6]:
println("Starting parametric study...\n")

for mesh_config in mesh_configs
    for stepper_config in stepper_configs
        for tol_config in tolerance_configs
            
            config_name = "Mesh: $(mesh_config.name), Stepper: $(stepper_config.name), Tol: $(tol_config.name)"
            println("Running: " * config_name)
            
            # Run the simulation and time it
            elapsed_time = @elapsed begin
                sol, num_dofs = run_simulation(mesh_config.nels, stepper_config.stepper, T_end, dt0, tol_config.abstol, tol_config.reltol)
            end
            
            # Store results
            stats = sol.stats
            push!(results, (
                mesh = mesh_config.name,
                dofs = num_dofs,
                stepper = stepper_config.name,
                tolerance = tol_config.name,
                time_sec = round(elapsed_time, digits=2),
                naccept = stats.naccept,
                nreject = stats.nreject,
                nsteps = stats.naccept + stats.nreject,
                nf = stats.nf, # Number of function evaluations (RHS)
                njacs = stats.njacs, # Number of Jacobian evaluations
                nw = stats.nw, # Number of W-matrix factorizations (LU solves)
                retcode = sol.retcode
            ))
            println("Finished in $(round(elapsed_time, digits=2)) seconds. Result: $(sol.retcode)\n")
        end
    end
end

println("Parametric study complete.")

Starting parametric study...

Running: Mesh: Coarse (10x4), Stepper: Rodas5P, Tol: Low Accuracy


LoadError: MethodError: [0mCannot `convert` an object of type [92mtypeof(ferrite_limiter!)[39m[0m to an object of type [91mtypeof(OrdinaryDiffEqCore.trivial_limiter!)[39m

[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T
[0m[90m   @[39m [90mBase[39m [90m[4mBase.jl:84[24m[39m


## Section 7: Results and Analysis

The collected statistics are presented below in a DataFrame for easy comparison. Key metrics to observe are:

- **`time_sec`**: Wall-clock time for the entire simulation. This is our primary performance indicator.
- **`nsteps`**: Total number of time steps taken. Fewer steps for the same accuracy is generally better.
- **`nf`**, **`njacs`**, **`nw`**: Number of RHS evaluations, Jacobian evaluations, and linear solves. These show the internal workload of the solver.
- **`retcode`**: The solver's return code. `Success` indicates the simulation completed without issues.

In [ ]:
# Display results using a DataFrame
df = DataFrame(results)
display(df)

## Section 8: Brief Conclusion

This study provides a quantitative comparison of different numerical configurations for solving the transient Stokes problem. From the results, we can draw conclusions about:

1.  **Scalability with Mesh Size:** We can observe how the computation time and the number of solver iterations (`nw`) increase as the number of DOFs grows.
2.  **Solver Efficiency:** By comparing `Rodas5P`, `ROS34PW1a`, and `KenCarp4`, we can identify which implicit solver is most efficient for this specific type of problem (a stiff DAE arising from FEM).
3.  **Accuracy vs. Performance Trade-off:** The difference in `time_sec` and `nsteps` between the 'Low Accuracy' and 'High Accuracy' runs demonstrates the cost of requiring a more precise solution.

Overall, this analysis is crucial for selecting optimal numerical methods for larger and more complex fluid dynamics simulations.